In [19]:
import pandas as pd
import numpy as np
import sqlalchemy
import matplotlib.pyplot as plt
import plotly.express as px
from pathlib import Path
from textblob import TextBlob

In [29]:
DATA_RAW   = Path("./data_raw")
DATA_CLEAN = Path("./data_clean")
DATA_CLEAN.mkdir(exist_ok=True)

In [27]:

def sniff_sep(path):
    import csv
    sample = Path(path).read_text(encoding="utf-8", errors="replace")[:2048]
    try:
        dialect = csv.Sniffer().sniff(sample, delimiters=[',',';','\t','|'])
        return dialect.delimiter
    except:
        return ','

# 1. Load raw CSVs / Excels

In [ ]:
# waiting_times.csv  → raw_waits
sep = sniff_sep(DATA_RAW/"waiting_times.csv")
waits = pd.read_csv(DATA_RAW/"waiting_times.csv", sep=sep, encoding="utf-8", low_memory=False)
waits.columns = waits.columns.str.strip().str.upper()

In [ ]:
# link_attraction_park.csv → link
sep = sniff_sep(DATA_RAW / "link_attraction_park.csv")
link = pd.read_csv(DATA_RAW / "link_attraction_park.csv", sep=sep, encoding="utf-8", header=0)
link.columns = link.columns.str.strip().str.upper()

In [ ]:
sep = sniff_sep(DATA_RAW/"attendance.csv")
att = pd.read_csv(DATA_RAW/"attendance.csv", sep=sep, encoding="utf-8")
# Standardize all column names
att.columns = att.columns.str.strip().str.upper()
# Then rename the park column
att.rename(columns={'FACILITY_NAME': 'PARK'}, inplace=True)

# Quick check
print(att.columns.tolist())

In [ ]:
# disneylandreviews.csv → reviews
sep = sniff_sep(DATA_RAW / "disneylandreviews.csv")
reviews = pd.read_csv(DATA_RAW / "disneylandreviews.csv", sep=sep, encoding="cp1252")
reviews.columns = reviews.columns.str.strip().str.upper()

In [ ]:
# weather_data.csv → weather
sep = sniff_sep(DATA_RAW / "weather_data.csv")
weather = pd.read_csv(DATA_RAW / "weather_data.csv", sep=sep, encoding="utf-8")
weather.columns = weather.columns.str.strip().str.upper()

In [ ]:
# entity_schedule.csv (optional enrichment)
sep = sniff_sep(DATA_RAW / "entity_schedule.csv")
schedule = pd.read_csv(DATA_RAW / "entity_schedule.csv", sep=sep, encoding="utf-8")
schedule.columns = schedule.columns.str.strip().str.upper()

In [ ]:
# parade_night_show.xlsx (optional)
parades = pd.read_excel(DATA_RAW / "parade_night_show.xlsx", sheet_name=0)
parades.columns = parades.columns.str.strip().str.upper()

In [ ]:
# Trim whitespace
for df in (waits, link, att, reviews, weather, schedule, parades):
    for c in df.columns:
        # Only attempt strip on python str types
        df[c] = df[c].apply(lambda x: x.strip() if isinstance(x, str) else x)
        # Replace empty strings with NA
        df[c].replace('', pd.NA, inplace=True)

# Quick verification: show any columns where dtype is object but values are not strings
for df_name, df in zip(
    ["waits", "link", "att", "reviews", "weather", "schedule", "parades"],
    (waits, link, att, reviews, weather, schedule, parades)
):
    non_str = df.select_dtypes(include=['object']).map(lambda x: not isinstance(x, str) and pd.notna(x))
    count_non_str = non_str.sum().sum()
    print(f"{df_name}: {count_non_str} non-string entries in object columns")

In [ ]:
# waiting-times: build datetime & rename
waits["WORK_DATE"] = pd.to_datetime(waits["WORK_DATE"])
waits["HOUR"]      = waits["DEB_TIME_HOUR"].astype(int)
waits["DATETIME"]  = waits["WORK_DATE"] + pd.to_timedelta(waits["HOUR"], unit="h")
waits["WAIT_MIN"]  = waits["WAIT_TIME_MAX"].astype("Int16")


# 3. Build dim_park & dim_ride

In [44]:
# dim_park
dim_park = (
    link[["PARK"]]
    .drop_duplicates()
    .reset_index(drop=True)
    .reset_index()
    .rename(columns={"index":"PARK_ID"})
    .assign(PARK_ID=lambda d: d["PARK_ID"]+1)
)

In [45]:
# dim_ride
dim_ride = (
    link.merge(dim_park, on="PARK", how="left")
        .rename(columns={"ATTRACTION":"RIDE"})
        .drop_duplicates(subset=["RIDE","PARK_ID"])
        .reset_index(drop=True)
        .reset_index()
        .rename(columns={"index":"RIDE_ID"})
        .assign(RIDE_ID=lambda d: d["RIDE_ID"]+1)
)

In [46]:
# dim_date (calendar)
min_d = waits["DATETIME"].dt.floor("D").min()
max_d = waits["DATETIME"].dt.floor("D").max()
all_dates = pd.DataFrame({
    "DATE": pd.date_range(min_d, max_d, freq="D")
})
dim_date = all_dates.assign(
    DAY_OF_WEEK = all_dates["DATE"].dt.day_name(),
    MONTH_NAME  = all_dates["DATE"].dt.month_name(),
    MONTH_NUM   = all_dates["DATE"].dt.month,
    YEAR        = all_dates["DATE"].dt.year,
    WEEK_NUM    = all_dates["DATE"].dt.isocalendar().week,
    IS_WEEKEND  = all_dates["DATE"].dt.dayofweek.isin([5,6])
)

In [48]:
# 1. Add a DATE column to waits for the calendar join
waits['DATE'] = waits['DATETIME'].dt.floor('D')

# 2. Merge only with dim_ride and dim_date (dim_ride already contains PARK_ID)
fact_wait = (
    waits
    # bring in ride_id and park_id from dim_ride
    .merge(
        dim_ride[['RIDE_ID', 'RIDE', 'PARK_ID']],
        left_on='ENTITY_DESCRIPTION_SHORT',
        right_on='RIDE',
        how='left'
    )
    # bring in date dimensions
    .merge(
        dim_date[['DATE']],
        on='DATE',
        how='left'
    )
    # select only the final analytic columns
    [['RIDE_ID', 'PARK_ID', 'DATETIME', 'WAIT_MIN', 'CAPACITY', 'GUEST_CARRIED']]
    # drop any rows where the join failed
    .dropna(subset=['RIDE_ID', 'PARK_ID'])
)

# Quick sanity check
print(f"fact_wait rows: {len(fact_wait)}")
print(f"Unique rides mapped: {fact_wait['RIDE_ID'].nunique()}")

fact_wait rows: 3509324
Unique rides mapped: 39


In [55]:
# right after you load `att`
print(att.columns.tolist())

['USAGE_DATE', 'PARK', 'ATTENDANCE']


In [56]:
# 1. Standardize the column names
att.columns = att.columns.str.strip().str.upper()

# 2. Rename USAGE_DATE → DATE
att.rename(columns={'USAGE_DATE':'DATE'}, inplace=True)

# 3. Build the attendance fact
fact_attendance = (
    att
    .merge(dim_park, on="PARK", how="left")
    [['PARK_ID','DATE','ATTENDANCE']]
    .dropna(subset=['PARK_ID'])
)

print("fact_attendance rows:", len(fact_attendance))
print("Sample:")
print(fact_attendance.head())

fact_attendance rows: 2367
Sample:
   PARK_ID        DATE  ATTENDANCE
0        2  2018-06-01       46804
1        1  2018-06-01       20420
2        2  2018-06-02       57940
3        1  2018-06-02       29110
4        2  2018-06-03       44365


In [58]:
# Show me what columns you really have
print("reviews columns:", reviews.columns.tolist())

reviews columns: ['REVIEW_ID', 'RATING', 'YEAR_MONTH', 'REVIEWER_LOCATION', 'REVIEW_TEXT', 'BRANCH', 'SENTIMENT']


In [59]:
# 1) Rename BRANCH → PARK so it matches dim_park
reviews.rename(columns={'BRANCH': 'PARK'}, inplace=True)

# 2) Build a full REVIEW_DATE from YEAR_MONTH
#    e.g. '2020-05' → Timestamp('2020-05-01')
reviews['REVIEW_DATE'] = pd.to_datetime(
    reviews['YEAR_MONTH'].astype(str) + "-01",
    format="%Y-%m-%d",
    errors="coerce"
)

# 3) Quick check you now have the right columns
print("reviews cols:", reviews.columns.tolist())
print(reviews[['PARK','YEAR_MONTH','REVIEW_DATE']].head())

reviews cols: ['REVIEW_ID', 'RATING', 'YEAR_MONTH', 'REVIEWER_LOCATION', 'REVIEW_TEXT', 'PARK', 'SENTIMENT', 'REVIEW_DATE']
                  PARK YEAR_MONTH REVIEW_DATE
0  Disneyland_HongKong     2019-4  2019-04-01
1  Disneyland_HongKong     2019-5  2019-05-01
2  Disneyland_HongKong     2019-4  2019-04-01
3  Disneyland_HongKong     2019-4  2019-04-01
4  Disneyland_HongKong     2019-4  2019-04-01


In [60]:
fact_review = (
    reviews
    .merge(dim_park, on="PARK", how="left")
    .rename(columns={'REVIEW_ID': 'ID'})
    [['ID', 'PARK_ID', 'REVIEW_DATE', 'RATING', 'SENTIMENT']]
    .dropna(subset=['PARK_ID'])
)

print("fact_review rows:", len(fact_review))
fact_review.head()

fact_review rows: 0


,ID,PARK_ID,REVIEW_DATE,RATING,SENTIMENT


In [61]:
# 1) Unique parks in reviews
review_parks = set(reviews['PARK'].unique())
# 2) Parks in your dim_park
dim_parks    = set(dim_park['PARK'])

print("Review parks:", review_parks)
print("Known parks:", dim_parks)
print("Missing in dim_park:", review_parks - dim_parks)

Review parks: {'Disneyland_HongKong', 'Disneyland_California', 'Disneyland_Paris'}
Known parks: {'Tivoli Gardens', 'PortAventura World'}
Missing in dim_park: {'Disneyland_HongKong', 'Disneyland_California', 'Disneyland_Paris'}


# Persist the five clean tables

In [63]:
for name, df in [
    ("dim_park", dim_park),
    ("dim_ride", dim_ride),
    ("dim_date", dim_date),
    ("fact_wait", fact_wait),
    ("fact_attendance", fact_attendance),
    ("fact_review", fact_review)
]:
    df.to_parquet(DATA_CLEAN/f"{name}.parquet", index=False)
print("✅  All clean tables written to data_clean/ 🎉")

✅  All clean tables written to data_clean/ 🎉
